In [97]:
import pandas as pd
import numpy as n
from statsmodels.tsa.ar_model import AR
from statsmodels.api import OLS
import statsmodels.api as sm

# 2.2 Data

In [83]:
data_shampoo = pd.read_excel("Dataset/mindset_shampoo.xlsx")
data_shampoo

,time,consideration,liking,awareness,promotion,advertising,price,vol
0,1,14.260000,5.1360,26.780001,2.390000,449.571991,4.349,1280.439973
1,2,16.770000,5.0100,29.930000,4.900000,483.415985,4.406,1129.629970
2,3,17.170000,4.9010,29.629999,3.800000,374.109985,4.203,1059.380054
3,4,16.459999,5.0940,27.110001,2.940000,567.109985,4.145,933.610022
4,5,18.430000,4.9440,31.770000,2.810000,418.777008,4.212,826.359987
...,...,...,...,...,...,...,...,...
91,92,25.639999,5.6079,33.730000,50.020000,889.000000,4.340,3231.529951
92,93,22.639999,5.3838,32.139999,56.330002,1043.000000,4.260,2340.519905
93,94,19.900000,5.2489,29.650000,61.189999,1021.000000,4.130,3181.519985
94,95,18.010000,5.1457,28.129999,60.259998,923.000000,4.290,2530.339956


## 2.1 Potential

In [27]:
# Calculate potential for awareness and consideration

potential_awareness = (100 - np.array(data_shampoo['awareness']).mean()) / 100
potential_consideration = (100 - np.array(data_shampoo['consideration']).mean()) / 100

# Calculate potential for liking
data_shampoo['liking_transformed'] = data_shampoo['liking'] / 7
potential_liking = 1-np.array(data_shampoo['liking_transformed']).mean()

d = {1: ["awareness",potential_awareness],
2: ["liking", potential_liking],
3: ["consideration", potential_consideration]}

print ("{:<8} {:<15} {:<10}".format('  ','potential','value'))
for k, v in d.items():
    potent, val = v
    print ("{:<8} {:<15} {:<10}".format(k, potent, val)) 
print('We get that potential is {:2f} for awareness, {:2f} for liking, and {:2f} for consideration.'.format(potential_awareness,potential_liking,potential_consideration))

         potential       value     
1        awareness       0.7252697922786077
2        liking          0.26309612961042494
3        consideration   0.82520833303531
We get that potential is 0.725270 for awareness, 0.263096 for liking, and 0.825208 for consideration.


## 2.2 Stickness

In [54]:
model = AR(data_shampoo.awareness)
model_fitted = model.fit(ic='aic')
model_fitted_awareness_params = model_fitted.params.to_list()
model_fitted.params

const           6.394666
L1.awareness    0.452079
L2.awareness    0.045827
L3.awareness    0.268888
dtype: float64

In [64]:
model = AR(data_shampoo.liking)
model_fitted = model.fit(ic='aic')
model_fitted_liking_params = model_fitted.params.to_list()
model_fitted.params

const        0.589535
L1.liking    0.278069
L2.liking    0.077028
L3.liking    0.170588
L4.liking    0.037810
L5.liking   -0.174820
L6.liking   -0.067101
L7.liking    0.224286
L8.liking    0.163162
L9.liking    0.181459
dtype: float64

In [56]:
model = AR(data_shampoo.consideration)
model_fitted = model.fit(ic='aic')
model_fitted_consideration_params = model_fitted.params.to_list()
model_fitted.params

const               3.721656
L1.consideration    0.309113
L2.consideration    0.319946
L3.consideration    0.161963
dtype: float64

From the above results, we can calculate stickiness of mindset metrics by summing up their AR coefficients for each mindset metric:

In [65]:
stick_awareness = sum(model_fitted_awareness_params[1:])
stick_liking = sum(model_fitted_liking_params[1:])
stick_consideration =  sum(model_fitted_consideration_params[1:])

## 2.3 Responsiveness

In [85]:
data_shampoo_lag_aware = data_shampoo.awareness.shift(1)
data_shampoo_lag_aware[0] = 0
data_shampoo['lag_aware'] = data_shampoo_lag_aware

data_shampoo_lag_liking = data_shampoo.liking.shift(1)
data_shampoo_lag_liking[0] = 0
data_shampoo['lag_likin'] = data_shampoo_lag_liking

data_shampoo_lag_consideration = data_shampoo.consideration.shift(1)
data_shampoo_lag_consideration[0] = 0
data_shampoo['lag_consideration'] = data_shampoo_lag_consideration

data_shampoo_lag_sales = data_shampoo.vol.shift(1)
data_shampoo_lag_sales[0] = 0
data_shampoo['lag_sales'] = data_shampoo_lag_sales

data_shampoo

,time,consideration,liking,awareness,promotion,advertising,price,vol,lag_aware,lag_likin,lag_consideration,lag_sales
0,1,14.260000,5.1360,26.780001,2.390000,449.571991,4.349,1280.439973,0.000000,0.0000,0.000000,0.000000
1,2,16.770000,5.0100,29.930000,4.900000,483.415985,4.406,1129.629970,26.780001,5.1360,14.260000,1280.439973
2,3,17.170000,4.9010,29.629999,3.800000,374.109985,4.203,1059.380054,29.930000,5.0100,16.770000,1129.629970
3,4,16.459999,5.0940,27.110001,2.940000,567.109985,4.145,933.610022,29.629999,4.9010,17.170000,1059.380054
4,5,18.430000,4.9440,31.770000,2.810000,418.777008,4.212,826.359987,27.110001,5.0940,16.459999,933.610022
...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,25.639999,5.6079,33.730000,50.020000,889.000000,4.340,3231.529951,26.629999,5.7470,22.490000,1934.710026
92,93,22.639999,5.3838,32.139999,56.330002,1043.000000,4.260,2340.519905,33.730000,5.6079,25.639999,3231.529951
93,94,19.900000,5.2489,29.650000,61.189999,1021.000000,4.130,3181.519985,32.139999,5.3838,22.639999,2340.519905
94,95,18.010000,5.1457,28.129999,60.259998,923.000000,4.290,2530.339956,29.650000,5.2489,19.900000,3181.519985


In [100]:
def return_log_func(input_series):
    return_log = np.array(input_series)
    return_log = np.log(return_log+1)
    return return_log
    

data_shampoo['log_awareness'] = return_log_func(data_shampoo.awareness)
data_shampoo['log_consideration'] = return_log_func(data_shampoo.consideration)
data_shampoo['log_liking'] = return_log_func(data_shampoo.liking)
data_shampoo['log_sales'] = return_log_func(data_shampoo.vol)

data_shampoo['log_lag_aware'] = return_log_func(data_shampoo.lag_aware)
data_shampoo['log_lag_consideration'] = return_log_func(data_shampoo.lag_consideration)
data_shampoo['log_lag_liking'] = return_log_func(data_shampoo.lag_likin)
data_shampoo['log_lag_sales'] = return_log_func(data_shampoo.lag_sales)

data_shampoo['log_price'] = return_log_func(data_shampoo.price)

data_shampoo['log_promotion'] = return_log_func(data_shampoo.promotion)

data_shampoo['log_advertising'] = return_log_func(data_shampoo.advertising)

data_shampoo

,time,consideration,liking,awareness,promotion,advertising,price,vol,lag_aware,lag_likin,...,log_lag_aware,log_price,log_promotion,log_advertising,log_consideration,log_liking,log_sales,log_lag_consideration,log_lag_liking,log_lag_sales
0,1,14.260000,5.1360,26.780001,2.390000,449.571991,4.349,1280.439973,0.000000,0.0000,...,0.000000,1.676910,1.220830,6.110518,2.725235,1.814173,7.155740,0.000000,0.000000,0.000000
1,2,16.770000,5.0100,29.930000,4.900000,483.415985,4.406,1129.629970,26.780001,5.1360,...,3.324316,1.687509,1.774952,6.182944,2.877512,1.793425,7.030530,2.725235,1.814173,7.155740
2,3,17.170000,4.9010,29.629999,3.800000,374.109985,4.203,1059.380054,29.930000,5.0100,...,3.431727,1.649235,1.568616,5.927219,2.899772,1.775122,6.966383,2.877512,1.793425,7.030530
3,4,16.459999,5.0940,27.110001,2.940000,567.109985,4.145,933.610022,29.629999,4.9010,...,3.421980,1.638025,1.371181,6.342315,2.859912,1.807305,6.840129,2.899772,1.775122,6.966383
4,5,18.430000,4.9440,31.770000,2.810000,418.777008,4.212,826.359987,27.110001,5.0940,...,3.336125,1.650964,1.337629,6.039724,2.966818,1.782382,6.718240,2.859912,1.807305,6.840129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,25.639999,5.6079,33.730000,50.020000,889.000000,4.340,3231.529951,26.629999,5.7470,...,3.318902,1.675226,3.932218,6.791221,3.282414,1.888266,8.081020,3.156575,1.909098,7.568229
92,93,22.639999,5.3838,32.139999,56.330002,1043.000000,4.260,2340.519905,33.730000,5.6079,...,3.547604,1.660131,4.048824,6.950815,3.162940,1.853764,7.758556,3.282414,1.888266,8.081020
93,94,19.900000,5.2489,29.650000,61.189999,1021.000000,4.130,3181.519985,32.139999,5.3838,...,3.500741,1.635106,4.130194,6.929517,3.039749,1.832405,8.065429,3.162940,1.853764,7.758556
94,95,18.010000,5.1457,28.129999,60.259998,923.000000,4.290,2530.339956,29.650000,5.2489,...,3.422633,1.665818,4.115127,6.828712,2.944965,1.815753,7.836504,3.039749,1.832405,8.065429


In [98]:
X = data_shampoo[['log_lag_aware', 'log_price','log_promotion','log_advertising']] 
y = data_shampoo['log_awareness']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          log_awareness   R-squared:                       0.427
Model:                            OLS   Adj. R-squared:                  0.402
Method:                 Least Squares   F-statistic:                     16.97
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           1.98e-10
Time:                        22:51:38   Log-Likelihood:                 77.882
No. Observations:                  96   AIC:                            -145.8
Df Residuals:                      91   BIC:                            -132.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               2.9509      0.554      5.331      0.000       1.851       4.051
log_lag_aware       0.0467      0.032      1.446      0.152      -0.017       0.111
log_price          -0.0830      0.331     -0.250      0.803      -0.741       0.575
log_promotion       0.0215      0.015      1.458      0.148      -0.008       0.051
log_advertising     0.0521      0.008      6.729      0.000       0.037       0.067
==============================================================================
Omnibus:                       13.507   Durbin-Watson:                   1.448
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               14.501
Skew:                          -0.883   Prob(JB):                     0.000710
Kurtosis:                       3.710   Cond. No.                         444.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [103]:
X = data_shampoo[['log_lag_liking', 'log_price','log_promotion','log_advertising']] 
y = data_shampoo['log_liking']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             log_liking   R-squared:                       0.329
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     11.16
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           2.08e-07
Time:                        22:57:29   Log-Likelihood:                 195.26
No. Observations:                  96   AIC:                            -380.5
Df Residuals:                      91   BIC:                            -367.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               1.4541      0.161      9.037      0.000       1.134       1.774
log_lag_liking     -0.0016      0.018     -0.089      0.929      -0.038       0.035
log_price           0.1789      0.097      1.843      0.069      -0.014       0.372
log_promotion       0.0192      0.004      4.434      0.000       0.011       0.028
log_advertising     0.0024      0.002      1.053      0.295      -0.002       0.007
==============================================================================
Omnibus:                        3.893   Durbin-Watson:                   1.317
Prob(Omnibus):                  0.143   Jarque-Bera (JB):                2.387
Skew:                           0.167   Prob(JB):                        0.303
Kurtosis:                       2.303   Cond. No.                         411.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [105]:
X = data_shampoo[['log_lag_consideration', 'log_price','log_promotion','log_advertising']] 
y = data_shampoo['log_consideration']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      log_consideration   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     11.79
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           9.33e-08
Time:                        22:58:30   Log-Likelihood:                 59.041
No. Observations:                  96   AIC:                            -108.1
Df Residuals:                      91   BIC:                            -95.26
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.8628      0.654      2.846      0.005       0.563       3.163
log_lag_consideration     0.0996      0.044      2.253      0.027       0.012       0.187
log_price                 0.2801      0.401      0.698      0.487      -0.517       1.077
log_promotion             0.0662      0.018      3.622      0.000       0.030       0.102
log_advertising           0.0177      0.009      1.883      0.063      -0.001       0.036
==============================================================================
Omnibus:                        1.577   Durbin-Watson:                   1.409
Prob(Omnibus):                  0.454   Jarque-Bera (JB):                1.071
Skew:                          -0.229   Prob(JB):                        0.585
Kurtosis:                       3.239   Cond. No.                         425.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [107]:
X = data_shampoo[['log_lag_sales', 'log_price','log_promotion','log_advertising']] 
y = data_shampoo['log_sales']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_sales   R-squared:                       0.590
Model:                            OLS   Adj. R-squared:                  0.572
Method:                 Least Squares   F-statistic:                     32.73
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           6.73e-17
Time:                        22:59:07   Log-Likelihood:                -13.027
No. Observations:                  96   AIC:                             36.05
Df Residuals:                      91   BIC:                             48.88
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               6.2350      1.411      4.420      0.000       3.433       9.037
log_lag_sales       0.0642      0.039      1.647      0.103      -0.013       0.142
log_price          -0.2776      0.852     -0.326      0.745      -1.969       1.414
log_promotion       0.2970      0.041      7.214      0.000       0.215       0.379
log_advertising     0.0596      0.020      2.989      0.004       0.020       0.099
==============================================================================
Omnibus:                        2.721   Durbin-Watson:                   0.906
Prob(Omnibus):                  0.256   Jarque-Bera (JB):                2.381
Skew:                          -0.135   Prob(JB):                        0.304
Kurtosis:                       3.723   Cond. No.                         574.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Now we summarize our estimation results here in the below table:

## 2.4 Conversion

In [108]:
def return_log_func(input_series):
    return_log = np.array(input_series)
    return_log = np.log(return_log)
    return return_log
    

data_shampoo['log_awareness'] = return_log_func(data_shampoo.awareness)
data_shampoo['log_consideration'] = return_log_func(data_shampoo.consideration)
data_shampoo['log_liking'] = return_log_func(data_shampoo.liking)


X = data_shampoo[['log_lag_sales', 'log_awareness','log_consideration','log_liking']] 
y = data_shampoo['log_sales']
X = sm.add_constant(X)
est = sm.OLS(y, X).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_sales   R-squared:                       0.562
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     29.14
Date:                Thu, 25 Mar 2021   Prob (F-statistic):           1.35e-15
Time:                        23:05:44   Log-Likelihood:                -16.241
No. Observations:                  96   AIC:                             42.48
Df Residuals:                      91   BIC:                             55.30
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -1.8907      1.246     -1.517      0.133      -4.366       0.585
log_lag_sales         0.1121      0.038      2.925      0.004       0.036       0.188
log_awareness         0.8010      0.253      3.167      0.002       0.299       1.303
log_consideration     0.4267      0.304      1.405      0.163      -0.177       1.030
log_liking            2.8141      0.997      2.821      0.006       0.833       4.795
==============================================================================
Omnibus:                        0.035   Durbin-Watson:                   1.458
Prob(Omnibus):                  0.983   Jarque-Bera (JB):                0.155
Skew:                           0.037   Prob(JB):                        0.926
Kurtosis:                       2.817   Cond. No.                         450.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Summarizing model results, we get:

## 2.5 Result Summary

Now we are finally done with all the steps for analysis. We can aggregate results from each stage of estimation in the following table so that it is easier to make decisions.

# 3. Managerial Implication–What-if Analysis

## 3.1 Fivefolding Advertising Expense

### 3.1.1 New Values

In [114]:
new_awareness = round(0.275 * (500/100)**(0.0490) ,3)
new_consideration = round(0.175 * (500/100) **(0.0154),3)
new_liking = round(0.737 * (500/100) ** (0.0025),3)
new_sales = round(1800.391 * (500/100) ** (0.0578),3)

### 3.1.2 Gain (Short Run)

In [116]:
gain_awareness = round((new_awareness/0.275)   -1,3)
gain_consideration = round((new_consideration/0.175)-1,3)
gain_liking = round( (new_liking/0.737)-1,3)
gain_sales = round((new_sales/1800.391)-1,3)

### 3.1.3 Gain (Long Run)

In [117]:
lrgain_awareness = round(gain_awareness/(1-0.0921),3)
lrgain_consideration = round(gain_consideration/(1-0.2081),3)
lrgain_liking = round(gain_liking/(1-0.0164),3)
lrgain_sales = round(gain_sales/(1-0.1115),3)

### 3.1.4 Conversion

In [118]:
conversion_awareness = round(lrgain_awareness * 0.7630,3)
conversion_consideration = round(lrgain_consideration * 0.3140,3)
conversion_liking = round(lrgain_liking * 2.9632,3)